#**주차 수요 예측**
# 단계1 : 데이터 전처리

## 0.미션

단지별 등록 차량 수를 예측하고자 합니다. 이에 맞게 데이터프레임을 생성하시오.
* 1) 단지별 데이터와 상세 데이터를 분리합니다.
* 2) 단지별 데이터
    * 범주의 수 줄이기 : 너무 종류가 많은 범주는 의미상 적절하게 병합합니다. 
* 3) 상세 데이터를 단지별로 집계해야 합니다. 
    * 전용면적 : 구간으로 나누어 집계하기
    * 임대보증금 혹은 임대료 : 전용면적별 세대수를 감안하여 집계하기.
* 4) 단지별 데이터와 집계데이터를 하나로 합칩니다.
* 5) (옵션)추가 변수 만들기
    * 등록 차량수를 예측하기 위해 필요한 변수를 추가해 봅시다.

## 1.환경설정

* 세부 요구사항
    - 경로 설정 : 다음의 두가지 방법 중 하나를 선택하여 폴더를 준비하고 데이터를 로딩하시오.
        * 1) 로컬 수행(Ananconda)
            * 제공된 압축파일을 다운받아 압축을 풀고
            * anaconda의 root directory(보통 C:/Users/< ID > 에 project 폴더를 만들고, 복사해 넣습니다.
        * 2) 구글콜랩
            * 구글 드라이브 바로 밑에 project 폴더를 만들고, 
            * 데이터 파일을 복사해 넣습니다.

    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다. 
        * 필요하다고 판단되는 라이브러리를 추가하세요.


### (1) 경로 설정

#### 1) 로컬 수행(Anaconda)
* project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [ ]:
path = ''

#### 2) 구글 콜랩 수행

* 구글 드라이브 연결

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# path = '/content/drive/MyDrive/project/'

### (3) 라이브러리 불러오기

#### 1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import *

#### 2) 한글 폰트 설정


In [ ]:
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

### (4) 데이터 불러오기

* 주어진 데이터셋
    * train.xlsx : 학습 및 검증용
    * test.xlsx : 테스트용

#### 1) 데이터로딩

In [ ]:
file1 = 'train.xlsx'
file2 = 'test.xlsx'

In [ ]:
data = pd.read_excel(path + file1)
test = pd.read_excel(path + file2)

#### 2) 기본 정보 조회

In [ ]:
#전체 데이터의 행,열 개수 확인
data.shape

In [ ]:
#전체 데이터의 상위 5개 행 확인
data.head()

In [ ]:
#전체 데이터의 수치형 변수 분포 확인
data.describe()

In [ ]:
#전체 데이터의 모든 변수 확인
data.info()

## 2.데이터 전처리①

* **세부 요구사항**
    * NaN 조치
        * NaN은 조회합니다.
        * 준공일자만 여기서 조치합시다.
            * 준공연도만 필요합니다. 문자열로 변환한 후 앞 4자리를 잘라 준공연도로 변환합니다.
            * 잘못 들어간 데이터들과 nan 문자를 준공연도 최소값으로 치환합니다.
        * 나머지 변수들은 탐색 이후에 조치를 취합니다.
    * 불필요한 데이터 제거


### (1) NaN 조치

In [ ]:
data0 = data.copy()

In [ ]:
data0.isna().sum()

In [ ]:
data0['준공연도'] = data0['준공일자'].astype('str').str.slice(0, 4)
data0.loc[data0['준공연도'].isin(['nan', '1900','1111']), '준공연도'] = '1992'
data0['준공연도'] = data0['준공연도'].astype('int')

### (2) 불필요한 칼럼 제거

* **세부 요구사항**
    * 단지명 : 분석단위로 볼때 단일값.
    * 단지내주차면수 : 
        * 본 문제는 등록차량수를 예측하고, 그것을 기반으로 주차면수를 정하는 것이 목적 입니다. 
        * 그런데, 역으로 주차면수 먼저 정한후 그것을 기반으로 등록차량수를 예측하는 것은 성립될 수 없습니다.

* 불필요한 정보 제거하기

In [ ]:
vars = ['단지내주차면수', '단지명', '준공일자']
data0.drop(vars, axis = 1, inplace = True)

In [ ]:
data0.head()

## 3.데이터 전처리②

* **세부 요구사항**

    * 1) 데이터프레임을 두가지 형태로 나눕니다.

    * 2) 상세 데이터 집계하기

    * 3) 단지별 데이터와 상세 데이터 집계 결과 merge 시키기



### (1) 데이터프레임 두개로 나누기

* **세부 요구사항**
    * 단지별 데이터
        * 단지코드, 총세대수, 지역, 준공일자, 건물형태, 난방방식, 승강기설치여부, 실차량수
        * 단지별 데이터를 분할 한 후, 중복행을 제거합니다.
        * 중복행이 잘 제거 되었는지 확인합니다.
    * 상세 데이터
        * 단지코드, 전용면적별세대수,전용면적, 공급면적(공용),임대보증금,임대료

In [ ]:
vars = ['단지코드', '총세대수', '지역', '준공연도', '건물형태', '난방방식', '승강기설치여부', '실차량수']
data01 = data0[vars].copy()
data01 = data01.drop_duplicates()
data01.head()

In [ ]:
# 확인 : 단지 코드가 유니크해야 함.
temp = data01.groupby('단지코드', as_index = False)['총세대수'].count()
temp.columns = ['단지코드',	'count']
temp.loc[temp['count']>1] # 위 칼럼으로 중복 행 없음.

* 상세 데이터

In [ ]:
vars2 = ['단지코드', '전용면적별세대수','전용면적', '공급면적(공용)','임대보증금','임대료']
data02 = data0[vars2].copy()
data02.head()

### (2) 상세 데이터 집계하기

* **세부 요구사항**
    * 전용면적 구간으로 나누기
        * 전용면적을 의미 있는 단위로 자른 칼럼 만들기 (pd.cut)
        * 구간별 세대 수 집계
        * pivot
    * 임대보증금 혹은 임대료 : 적절하게 집계하기.
        * 평균, 가중평균, 중앙값 중에서 고려하기.
    * 임대건물구분, 공급유형
        * 전체 면적을 계산합니다.(공급면적 * 세대수)
        * 임대건물구분 별, 면적 합계를 구한 후, 면적 비율을 계산합니다.
        * 역시, 공급 유형별, 면적 합계를 구한 후, 면적 비율을 계산합니다.

#### 1) 전용면적

* 전용면적을 의미있는 단위로 자른 칼럼 만들기 (pd.cut)

In [ ]:
temp = data02.loc[: , ['단지코드','전용면적','전용면적별세대수']]

print(temp['전용면적'].min(), temp['전용면적'].max())
sns.histplot(x = temp['전용면적'], bins = 12)
plt.show()

In [ ]:
temp['전용면적_cut'] = pd.cut(temp['전용면적'], bins = [10,30,40,50,60,70, 80, 200]
                          , labels = ['전용면적_10_30','전용면적_30_40','전용면적_40_50','전용면적_50_60','전용면적_60_70','전용면적_70_80', '전용면적_80_200'])
temp.head()

* 10 단위 칼럼별 집계

In [ ]:
temp.groupby(['단지코드', '전용면적_cut'], as_index = False)['전용면적별세대수'].sum()

In [ ]:
temp2 = temp.groupby(['단지코드', '전용면적_cut'], as_index = False)['전용면적별세대수'].sum()

* pivot

In [ ]:
temp3 = temp2.pivot(index='단지코드', columns='전용면적_cut', values='전용면적별세대수')
temp3.reset_index(inplace = True)
temp3.head()

#### 2) 임대보증금, 임대료 집계 하기(평균)

In [ ]:
temp4 = data02.groupby('단지코드', as_index = False)[['임대보증금','임대료']].mean()
temp4

#### 3) 단지별 총 면적 구하기

In [ ]:
data02['총면적'] = data02['전용면적'] * data02['전용면적별세대수']

In [ ]:
temp6 = data02.groupby(['단지코드'], as_index = False)['총면적'].sum()
temp6.head()

### (3) 합치기

* **세부 요구사항**
    * [단지별 데이터]를 기준으로 상세데이터로 만든 데이터셋을 하나씩 merge 합니다.
    * merge를 사용할 때, **how = 'left', on = '단지코드'** 옵션을 이용합니다.
        * [단지별 데이터]가 기준(left)입니다. 

* 단지별 데이터 + 전용면적별 세대수

In [ ]:
base_data = pd.merge(data01,temp6, how = 'left', on = '단지코드')
base_data = pd.merge(base_data,temp3, how = 'left', on = '단지코드')

In [ ]:
base_data.head()

* 평균 임대 보증금/임대료 합치기

In [ ]:
base_data = pd.merge(base_data,temp4, how = 'left', on = '단지코드')
base_data.head()

## 4.데이터셋 저장하기

* **세부 요구사항**
    * joblib.dump를 이용하시오.
    * 저장할 파일의 확장자는 보통 .pkl 입니다.

In [ ]:
import joblib

joblib.dump(base_data, path + 'base_data.pkl')